# Fine-tuning a Large Language Model

In this lecture we will be looking at how to fine-tune an existing pre-trained language model.

## Learning outcomes
* You will learn how to download a pre-trained model and a training dataset from Hugging Face.
* You will learn how to fine-tune the downloaded model with the dataset using Hugging Face trl library and the supervised fine-tuning (SFT) method.
* You will learn how to use the fine-tuned model to generate text based on user input / prompts.
* You will learn how to upload the fine-tuned model to your own Hugging Face repository so that it can be used later or shared with other users.

## Prerequistes
* You will need the following free accounts: Google, Hugging Face and Weights & Biases. You may use your existing accounts or create new accounts for the purposes of this course.
* We will use the [Hugging Face](https://huggingface.co/) libraries: transformers (for models), datasets (for datasets), trl (for training). We will also store the fine-tuned models in a Hugging Face repository.
* Training is done using [Google Colab](https://colab.research.google.com/), which provides free access to Jupyter notebooks backed with a GPU compute required for fine-tuning.
* For monitoring the training run we will use [Weights & Biases](https://wandb.ai/)


## Fine-tuning

Let's first install some pre-requisites using Python's package manager pip

In [1]:
!pip install transformers peft accelerate datasets trl wandb bitsandbytes

Then we need to import the required libraries

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TextStreamer, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from datasets import load_dataset
from trl import SFTTrainer
import huggingface_hub
import torch
import wandb
from google.colab import userdata


We will download a pre-trained large language model from Hugging Face and a dataset to train the model with. Below we assign these to variables we will use later. We will also set the name of the repository and model for the fine-tuned model.

In [3]:
# Pre trained model
model_name = "Qwen/Qwen3-4B"

# Dataset name
dataset_name = "amayuelas/aya-global-exams-spanish"

HUGGING_FACE_USERNAME = "IvanInRainbows"  # <---- change to your hugging face username

# Hugging face repository link to save fine-tuned model(Create new repository in huggingface,copy and paste here)
new_model = f"{HUGGING_FACE_USERNAME}/Qwen3-4B-Finetuned"

To access your Hugging Face account, you need to log in. First go to your Hugging Face account, click *Settings* and select *Access Tokens*. Create a new token and copy the token. Then execute the below login command and when asked paste an access token.  

In [4]:
huggingface_hub.login(token=userdata.get('HF'))

Let's then download a subset of the dataset we want to use. Below we limit the dataset to the first 10,000 examples in order to save time. In real life you would probably use the full dataset.

In [10]:
# Load a small subset of the instruction-tuning dataset
raw_dataset = load_dataset(dataset_name, split="train[:10000]")

def format_example(example):
    # Turn the Alpaca-style fields into a single text field
    return {
        "text": f"### Question:\n{example['question']}\n\n### Options:\n{example['options']}\n\n### Answer:\n{example['answer']}"
    }

# Map to a simple {'text': ...} format and keep a tiny subset so it trains quickly
dataset = raw_dataset.map(format_example)
dataset = dataset.shuffle(seed=42).select(range(50))
dataset["text"][0]


"### Question:\n¿Qué es FTP?\n\n### Options:\n['Un protocolo para conectar ordenadores a Internet.', 'Un protocolo de transferencia de ficheros.', 'Un protocolo de envío de correo.', 'Un tipo de servidor web.']\n\n### Answer:\n2"

Let's then download the model. We first create a config object for quantization of the model using bitsandbytes. Bitsandbytes enables accessible large language models via k-bit quantization for PyTorch.

We also need to download the tokenizer.

In [13]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)
print("success")
model = prepare_model_for_kbit_training(model)
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
#tokenizer.bos_token, tokenizer.add_eos_token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

success


In [14]:
tokenizer.bos_token, tokenizer.add_eos_token

(None, True)

Below we log in to Weights & Biases for experiment tracking.

> * In Colab, store your key in the `WANDB_API_KEY` environment variable, or  
> * Call `wandb.login()` and paste the key interactively when prompted.
>
> You can find your key in your [Weights & Biases account](https://wandb.ai/).


In [18]:
# Monitoring login (uses the WANDB_API_KEY environment variable if set)
wandb.login(key=userdata.get('wandb'))
run = wandb.init(project="llm-finetuning-demo", job_type="training", anonymous="allow")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Then we'll create a configuration for the lo-rank adaptation method we will use.

In [19]:
peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

#### LoRA Target Modules

LoRA adds small trainable matrices into selected linear layers of a transformer.
**Target modules** tell LoRA *which* layers to modify.

**Common module names (LLaMA / Mistral / Qwen)**

**Attention layers**

* **q_proj**: creates attention *queries*
* **k_proj**: creates attention *keys*
* **v_proj**: creates attention *values*
* **o_proj**: attention outputs

**Feed-forward (MLP) layers**

* **gate_proj**: gating in SwiGLU
* **up_proj**: expands hidden size
* **down_proj**: reduces back to model size

**Recommended set for most models**

```python
["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
```

**If VRAM is tight (e.g., T4)**

```python
["q_proj", "k_proj", "v_proj", "o_proj"]
```

These layers give the best trade-off between memory use and performance.


We need to set the training arguments for the training run.

In [20]:
training_arguments = TrainingArguments(
    output_dir="./results",          # Where to save checkpoints & logs
    num_train_epochs=1,              # Number of full passes through the dataset
    per_device_train_batch_size=8,   # Batch size per GPU (before gradient accumulation)
    gradient_accumulation_steps=2,   # Accumulate gradients to simulate a larger batch (8×2 = 16)
    optim="paged_adamw_8bit",        # Memory-efficient optimizer from bitsandbytes (QLoRA-friendly)
    save_steps=1000,                 # Save model every 1000 steps (set high to avoid slowing training)
    logging_steps=10,                # Log metrics to W&B every 10 steps
    learning_rate=2e-4,              # Base learning rate for training
    weight_decay=0.001,              # Regularization to reduce overfitting
    fp16=False,                      # Use float16 (disabled here)
    bf16=False,                      # Use bfloat16 (disable on GPUs like T4 that don't support it)
    max_grad_norm=0.3,               # Gradient clipping for training stability
    max_steps=-1,                    # Train for full epochs (no manual step limit)
    warmup_ratio=0.3,                # Fraction of steps for LR warmup (30%)
    group_by_length=True,            # Buckets sequences by length for efficiency
    lr_scheduler_type="linear",      # Linear learning-rate schedule
    report_to="wandb",               # Send logs to Weights & Biases
)


Finally we create the trainer object that uses supervised fine-tuning (SFT) as the training method.

In [21]:
# Setting SFT parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_arguments,
    processing_class=tokenizer,
)

Adding EOS to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Then, we can execute the training run.

In [22]:
# Train model
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


TrainOutput(global_step=4, training_loss=2.3123860359191895, metrics={'train_runtime': 32.2824, 'train_samples_per_second': 1.549, 'train_steps_per_second': 0.124, 'total_flos': 176609998909440.0, 'train_loss': 2.3123860359191895, 'entropy': 1.1270181962421961, 'num_tokens': 5773.0, 'mean_token_accuracy': 0.5846932785851615, 'epoch': 1.0})

In [23]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

train/entropy,▁
train/epoch,▁
train/global_step,▁
train/mean_token_accuracy,▁
train/num_tokens,▁
total_flos,176609998909440.0
train/entropy,1.12702
train/epoch,1
train/global_step,4
train/mean_token_accuracy,0.58469
train/num_tokens,5773


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2560)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2560, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2560, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
       

In [24]:
def stream(user_prompt: str):
    # Put model in eval mode
    model.eval()

    # Works even with device_map="auto"
    device = next(model.parameters()).device

    system_prompt = (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
    )
    B_INST, E_INST = "### Instruction:\n", "\n\n### Response:\n"
    prompt = f"{system_prompt}{B_INST}{user_prompt.strip()}{E_INST}"

    # Move inputs to the same device as the model
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Stream tokens directly to notebook output
    streamer = TextStreamer(
        tokenizer,
        skip_prompt=True,          # don't print the full prompt
        skip_special_tokens=True,
    )

    with torch.inference_mode():
        _ = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            streamer=streamer,
            eos_token_id=tokenizer.eos_token_id,
        )

In [26]:
stream("""
El Estatuto Básico del Empleado Público establece que para poder participar en los procesos de promoción interna los
1. funcionarios deberán poseer, entre otros requisitos, una antigüedad en el Subgrupo inferior, o Grupo de clasificación
2. profesional, en el supuesto de que éste no tenga Subgrupo de, al menos:
3. Depende del Subgrupo o Grupo al que se pretenda acceder.
4. Tres años continuados en el puesto desde el que se presenta a la promoción interna.
""")

The Basic Statute of the Public Employee establishes that to be eligible for internal promotion processes, public employees must meet certain requirements. Specifically, for the promotion to a higher group or sub-group, they must have at least a certain number of years of service in the lower sub-group or professional group. The exact number of years required depends on the sub-group or group they are currently in. For example, if an employee is in a lower sub-group, they must have at least three consecutive years in the position from which they are applying for promotion. This requirement ensures that employees have sufficient experience and stability in their current role before being considered for a promotion to a higher level.
The Basic Statute of the Public Employee establishes that to be eligible for internal promotion processes, public employees must meet certain requirements. Specifically, for the promotion to a higher group or sub-group, they must have at least a certain numb

In [27]:
# Same bnb_config as above
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

model = PeftModel.from_pretrained(base_model, new_model)

# Try merging LoRA into the base model
model = model.merge_and_unload()  # may still be heavy on T4 depending on model size

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
model.push_to_hub(new_model)
tokenizer.push_to_hub(new_model)